# Semantic Classification

Semantic classification is a machine learning technique that categorizes text based on its meaning using vector embeddings and similarity matching. This approach offers a compelling alternative to using large language models (LLMs) for classification tasks.

## Semantic Classification vs. LLM Classification

**LLM-based classification** involves sending text to models like GPT-4 or Claude with prompts asking them to categorize content. While powerful, this approach has several limitations:
- **Cost**: API calls for every classification can be expensive at scale
- **Latency**: Network requests and model inference add delay

**Semantic classification** uses vector embeddings to represent text meaning numerically, then applies similarity thresholds to determine categories:
- **Speed**: Near-instantaneous vector similarity calculations
- **Cost-effective**: No API costs after initial setup

## How It Works

Text is converted into high-dimensional vectors that capture semantic meaning. Reference examples for each category are embedded, and new content is classified by measuring vector similarity. Content that falls within a defined distance threshold of category references gets classified accordingly.

In this notebook, we'll build a semantic classifier using RedisVL to identify AI-related content, demonstrating an efficient alternative to LLM-based classification.

## Installing dependencies

We'll need two main libraries for this semantic classification example:

**RedisVL** - A Python library that provides vector database capabilities and semantic routing functionality on top of Redis. RedisVL handles the vector storage, similarity search, and routing logic that powers our semantic classifier.

**HuggingFace Sentence Transformers** - A library that provides pre-trained models for converting text into high-quality sentence embeddings. These models have been specifically trained to capture semantic meaning, making them ideal for similarity-based classification tasks.

The sentence transformers library will automatically download the embedding model on first use. We're using a lightweight model (`all-MiniLM-L6-v2`) that provides good performance while being fast enough for real-time classification.

In [ ]:
%pip install -q "redisvl>=0.8.2" sentence-transformers

## Setting up the Text Vectorizer

The vectorizer is responsible for converting text into numerical vector representations that capture semantic meaning. RedisVL provides several vectorizer options such as OpenAI and VertexAI. We're using the HuggingFace Text Vectorizer for this example.

In [ ]:
from redisvl.utils.vectorize import HFTextVectorizer

vectorizer = HFTextVectorizer()

## Loading references
In this recipe, we're trying to classify posts that are related to artificial intelligence. In order to do so, we will vectorize a couple of hundred examples that have been previously generated for us.

In [ ]:
with open('1_references.txt', 'r') as file:
    artificial_intelligence_references = [line.strip() for line in file.readlines()]

artificial_intelligence_references[:10]

## Creating a route

Now, let's create a route. A route is a classification category that defines what types of content should be grouped together based on semantic similarity. Each route contains:

- **Reference examples**: Sample text that represents the category you want to classify
- **Distance threshold**: How similar new text must be to the references to match the route
- **Route name**: An identifier for this classification category

In [ ]:
from redisvl.extensions.router import Route, SemanticRouter

artificial_intelligence_route = Route(
    name="artificial_intelligence_references",
    references=artificial_intelligence_references,
    distance_threshold=0.7
)

## Creating the Semantic Router

The SemanticRouter is the central component that orchestrates the classification process. It combines your routes, vectorizer, and Redis storage to provide fast semantic classification capabilities.


In [ ]:
classifier_router = SemanticRouter(
    name="classifier",
    vectorizer=vectorizer,
    routes=[artificial_intelligence_route],
    overwrite=False,
)

## Testing our semantic classification solution

In [ ]:
user_query = "Redis is a great tool for building applied AI systems because it works well as agent memory"

route_match = classifier_router(user_query)

route_match

In [ ]:
user_query = "Bavaria is a nice place to visit"

route_match = classifier_router(user_query)

route_match